<a href="https://colab.research.google.com/github/quanganh1999/NF-Net-on-CIFAR/blob/main/Train_NF_ResNet_on_Cifar_100_using_PyTorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⚙️ Imports and Setups

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/NF-Resnet

In [ ]:
%%capture
# Install pytorch lighting
!pip install pytorch-lightning --quiet
# Install weights and biases
!pip install wandb --quiet

In [ ]:
!pip install lightning-bolts["extra"] --quiet

In [ ]:
!git clone https://github.com/rwightman/pytorch-image-models

In [ ]:
!pip install git+https://github.com/rwightman/pytorch-image-models.git --quiet

In [ ]:
# regular imports
import sys
sys.path.append("pytorch-image-models")
import os
import re
import numpy as np

# pytorch related imports 
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url

# import for nfnet
import timm

# lightning related imports
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# sklearn related imports
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize

# import wandb and login
import wandb
wandb.login()

In [ ]:
#@title
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100
from pytorch_lightning.utilities.seed import seed_everything
import pl_bolts

In [ ]:
seed_everything(1999)

## 🎨 Using DataModules - `Clatech101DataModule`

DataModules are a way of decoupling data-related hooks from the `LightningModule` so you can develop dataset agnostic models.

In [ ]:
class CIFAR100Data(pl.LightningDataModule):
      def __init__(self, batch_size, data_dir: str = './'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size        
        self.mean = (0.4914, 0.4822, 0.4465)
        self.std = (0.2471, 0.2435, 0.2616)

        #augmentation (use other strong augmentation)
        # self.train_transform = transforms.Compose(
        #     [
        #         transforms.RandomCrop(32, padding=4),
        #         transforms.RandomHorizontalFlip(),
        #         transforms.ToTensor(),
        #         transforms.Normalize(self.mean, self.std),
        #     ]
        # )
        self.train_transform = transforms.Compose([
          transforms.RandomResizedCrop((256, 256), scale=(0.05, 1.0)),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])
   
        #need to resize for more acc ??
        # self.test_transform = transforms.Compose(
        #     [
        #         transforms.ToTensor(),
        #         transforms.Normalize(self.mean, self.std),
        #     ]
        # )
        self.test_transform = transforms.Compose([
        transforms.Resize((320, 320)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])
        # self.dims = (3, 32, 32)
        self.num_classes = 100

      def prepare_data(self):
        # download 
        CIFAR100(self.data_dir, train=True, download=True)
        CIFAR100(self.data_dir, train=False, download=True)    

      def setup(self, stage=None):        
        if stage == 'fit' or stage is None:
            # load the dataset    
            self.cifar_full_train = CIFAR100(self.data_dir, train=True, transform=self.train_transform)
            self.cifar_full_val = CIFAR100(self.data_dir, train=True, transform=self.test_transform)             
            num_train = len(self.cifar_full_train)
            indices = list(range(num_train))
            split = int(np.floor(0.1 * num_train))
            np.random.seed(1999)
            np.random.shuffle(indices)

            #train
            train_idx, valid_idx = indices[split:], indices[:split]
            self.train_sampler = SubsetRandomSampler(train_idx)
            self.valid_sampler = SubsetRandomSampler(valid_idx)            

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.cifar_test = CIFAR100(self.data_dir, train=False, transform=self.test_transform)        

      def train_dataloader(self):
        #return DataLoader(self.cifar_full_train, batch_size=self.batch_size, sampler=self.train_sampler)
        return DataLoader(self.cifar_full_train, batch_size=self.batch_size, shuffle=True)

      def val_dataloader(self):      
        #return DataLoader(self.cifar_full_val, batch_size=self.batch_size, sampler=self.valid_sampler)
        return DataLoader(self.cifar_test, batch_size=self.batch_size)

      def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size)

## 📲 Callbacks

#### 🚏 Earlystopping

In [ ]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   patience=3,
   verbose=False,
   mode='min'
)

#### 🛃 Custom Callback - `ImagePredictionLogger`

In [ ]:
class ImagePredictionLogger(Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.num_samples = num_samples
        self.val_imgs, self.val_labels = val_samples
        
    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
       
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)
        
        trainer.logger.experiment.log({
            "examples":[wandb.Image(x, caption=f"Pred:{pred}, Label:{y}") 
                           for x, pred, y in zip(val_imgs[:self.num_samples], 
                                                 preds[:self.num_samples], 
                                                 val_labels[:self.num_samples])]
            })

#### 💾 Model Checkpoint Callback

In [ ]:
MODEL_CKPT_PATH = './model'
# MODEL_CKPT = 'model-{epoch:02d}-{val_loss:.2f}'
MODEL_CKPT = 'model-{epoch:02d}-{val_acc:.3f}'

checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    dirpath = MODEL_CKPT_PATH,
    filename=MODEL_CKPT,
    save_top_k=3,
    save_last = True,
    mode='max')

## 🎺 Define The Model

In [ ]:
class LitModel(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4):
        super().__init__()
        
        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.dim = input_shape
        self.num_classes = num_classes

        #train from scratch
        # self.classifier = timm.create_model('nf_resnet50', pretrained=False, num_classes = num_classes)

        #transfer learning
        #fine-tuning
        self.model = timm.create_model("eca_nfnet_l1", pretrained=True, num_classes=num_classes) #256 input 320 test

        #feature exactor (freezee all layers except the last layer)
        # self.model = timm.create_model("eca_nfnet_l1", pretrained=True)        
        # for param in self.model.parameters():
        #     param.requires_grad = False
        # self.model.reset_classifier(num_classes)

    def forward(self, x):
      x = F.log_softmax(self.model.forward(x), dim=1)
      return x


    # logic for a single training step
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # training metrics
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        
        return loss

    # logic for a single validation step
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    # logic for a single testing step
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        # optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-6, nesterov = True)
        scheduler = pl_bolts.optimizers.lr_scheduler.LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs = 5, max_epochs = 60)
        # return optimizer
        return [optimizer], [scheduler]


## ⚡ Train and Evaluate The Model

In [ ]:
# Init our data pipeline
BATCH_SIZE = 64
# dm = CIFAR10Data(batch_size=BATCH_SIZE)
dm = CIFAR100Data(batch_size=BATCH_SIZE)
# To access the x_dataloader we need to call prepare_data and setup.
dm.prepare_data()
dm.setup()

# Samples required by the custom ImagePredictionLogger callback to log image predictions.
val_samples = next(iter(dm.val_dataloader()))
val_imgs, val_labels = val_samples[0], val_samples[1]
val_imgs.shape, val_labels.shape

In [ ]:
# Init our model
model = LitModel((3, 256, 256), 100)

# Initialize wandb logger
wandb_logger = WandbLogger(project='nfnet', job_type='train-nf-resnet')

# Initialize a trainer
trainer = pl.Trainer(max_epochs=60,
                     progress_bar_refresh_rate=5, 
                     gpus=1, 
                     logger=wandb_logger,
                     #early_stop_callback,
                     callbacks=[ImagePredictionLogger(val_samples), checkpoint_callback]
                     )

# Train the model ⚡🚅⚡
trainer.fit(model, dm) 

# Evaluate the model on the held out test set ⚡⚡
trainer.test()

# Close wandb run
wandb.finish()

In [ ]:
#resume from checkpoint:
model = LitModel((3, 256, 256), 10)

# Initialize wandb logger
wandb_logger = WandbLogger(project='nfnet', job_type='train-nf-resnet')

# Initialize a trainer
trainer = pl.Trainer(max_epochs=60,
                     progress_bar_refresh_rate=5, 
                     gpus=1, 
                     logger=wandb_logger,
                     #early_stop_callback,
                     callbacks=[ImagePredictionLogger(val_samples), checkpoint_callback],
                     resume_from_checkpoint='model/last.ckpt'
                     )

# Train the model ⚡🚅⚡
trainer.fit(model, dm) 

# Evaluate the model on the held out test set ⚡⚡
trainer.test()

# Close wandb run
wandb.finish()

In [ ]:
#Test with checkpoint
LitModel((3, 32, 32), 10)
model = LitModel.load_from_checkpoint(checkpoint_path='./model/model2-epoch=41-val_acc=0.974.ckpt')

# init trainer with whatever options
trainer = trainer = pl.Trainer(gpus=1)

# test (pass in the model)
trainer.test(model, datamodule = dm)

In [ ]:
%ls model/